# Instalação de bibliotecas

In [ ]:
!pip install tensorflow==2.6.0
!pip install keras==2.6.0
!pip install numpy==1.19.5 pandas
!pip install protobuf==3.19.0
!pip install scipy==1.7.3
!pip install motmetrics==1.4.0

#Importa bibliotecas

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd  # Importa o Pandas para manipulação e análise de dados em DataFrames
import tensorflow as tf  # Importa o TensorFlow para construção e treinamento de modelos de aprendizado de máquina
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, LeakyReLU  # Importa camadas específicas da Keras para criação de redes neurais
from tensorflow.keras.models import Sequential  # Importa o modelo sequencial para empilhar camadas na criação de modelos
from tensorflow.keras.optimizers import Adam  # Importa o otimizador Adam para ajuste dos pesos da rede neural
from tensorflow.keras.callbacks import EarlyStopping  # Importa EarlyStopping para interromper o treinamento caso não haja melhoria
from sklearn.model_selection import train_test_split  # Importa função para dividir os dados em treino e teste
from sklearn.preprocessing import MinMaxScaler  # Importa MinMaxScaler para normalizar os dados em uma escala entre 0 e 1
import numpy as np  # Importa o NumPy para operações numéricas e manipulação de arrays
import random  # Importa random para geração de números aleatórios, útil para controle de aleatoriedade
import os  # Importa os para manipulação de arquivos e diretórios no sistema
import motmetrics as mm  # Importa motmetrics para cálculo de métricas de rastreamento de objetos
from scipy.spatial.distance import cdist  # Importa cdist para cálculo de distâncias entre arrays, útil em associações de rastreamento
import itertools  # Importa itertools para criação de iteradores complexos, usado em operações combinatórias


# Define semente e carrega dataset filtrado

In [ ]:
# Definir seed para reprodutibilidade
SEED = 42  # Define um valor de semente fixa para garantir a reprodutibilidade dos resultados
np.random.seed(SEED)  # Define a seed para operações aleatórias do NumPy
tf.random.set_seed(SEED)  # Define a seed para operações aleatórias do TensorFlow
random.seed(SEED)  # Define a seed para operações aleatórias da biblioteca random
os.environ['PYTHONHASHSEED'] = '0'  # Define o hash do Python como constante para consistência nos resultados

# Carregar o arquivo CSV
path_filtered = 'https://drive.google.com/uc?id=1PBBHU8JvnOBBVeV0HcyLjcxyR8upZafX'  # Caminho do arquivo CSV armazenado no Google Drive
data = pd.read_csv(path_filtered)  # Carrega o arquivo CSV para um DataFrame usando o Pandas


# Preparação dos dados

In [ ]:
# Função para preparar os dados
def prepare_data_for_prediction(data, sequence_length, feature_cols, target_cols):
    # Ordena o DataFrame por instance_token, camera e timestamp para garantir a sequência correta
    data = data.sort_values(by=['instance_token', 'camera', 'timestamp']).reset_index(drop=True)

    # Aplica one-hot encoding na coluna 'camera'
    camera_dummies = pd.get_dummies(data['camera'], prefix='camera')  # Cria variáveis binárias para cada valor da coluna 'camera'
    data = pd.concat([data, camera_dummies], axis=1)  # Concatena as novas colunas ao DataFrame original

    # Atualiza feature_cols com as novas colunas
    feature_cols = feature_cols + list(camera_dummies.columns)  # Adiciona as colunas de 'camera' codificadas em 'feature_cols'

    # Inicializa listas para armazenar sequências, alvos e metadados
    sequences = []  # Lista para armazenar sequências de características
    targets = []  # Lista para armazenar os alvos
    metadata_list = []  # Lista para armazenar metadados associados a cada sequência

    # Agrupa dados por instance_token e camera
    grouped = data.groupby(['instance_token', 'camera'])  # Agrupa o DataFrame com base nas colunas instance_token e camera

    for (instance_token, camera), group in grouped:
        group = group.reset_index(drop=True)  # Reseta o índice do grupo para facilitar o acesso sequencial
        num_samples = len(group)  # Conta o número de amostras no grupo
        if num_samples <= sequence_length:
            continue  # Pula o grupo se não houver amostras suficientes para uma sequência completa
        for i in range(sequence_length, num_samples):
            seq_features = group.loc[i - sequence_length:i - 1, feature_cols].values  # Extrai as características da sequência
            target = group.loc[i, target_cols].values  # Define o alvo como a linha atual da sequência
            sequences.append(seq_features)  # Adiciona a sequência de características à lista de sequências
            targets.append(target)  # Adiciona o alvo à lista de alvos
            metadata_list.append({
                'instance_token': instance_token,
                'timestamp': group.loc[i, 'timestamp'],
                'camera': camera,
                'scene_token': group.loc[i, 'scene_token']
            })  # Adiciona o metadado associado à sequência atual

    # Converte listas para arrays numpy
    sequences = np.array(sequences)  # Converte a lista de sequências para um array NumPy
    targets = np.array(targets)  # Converte a lista de alvos para um array NumPy

    # Converte metadata_list em DataFrame
    metadata = pd.DataFrame(metadata_list)  # Converte a lista de metadados para um DataFrame para fácil acesso e manipulação

    # Garante que os tipos dos dados sejam float32
    sequences = sequences.astype(np.float32)  # Define os tipos das sequências como float32 para consistência
    targets = targets.astype(np.float32)  # Define os tipos dos alvos como float32 para consistência

    # Verifica se há valores NaN
    if np.isnan(sequences).any() or np.isnan(targets).any():
        print("Dados contêm NaNs. Por favor, verifique seus dados.")  # Informa ao usuário que há valores NaN
        raise ValueError("Dados contêm NaNs.")  # Lança um erro caso haja valores NaN nos dados

    # Verifica consistência dos comprimentos
    assert len(sequences) == len(targets) == len(metadata), "Inconsistência nos comprimentos das listas de saída."  # Garante que todas as listas tenham o mesmo comprimento

    return sequences, targets, metadata  # Retorna as sequências, alvos e metadados


# Divide os dados em treino e teste 80% e 20% respectivamente

In [ ]:
# Dividir os dados em treino e teste garantindo que cenas não sejam misturadas
unique_scenes = data['scene_token'].unique()  # Obtém uma lista única de cenas presentes na coluna 'scene_token'
train_scenes, test_scenes = train_test_split(unique_scenes, test_size=0.2, random_state=SEED)  # Divide as cenas em treino e teste, usando 20% para teste e garantindo reprodutibilidade com a seed

train_data = data[data['scene_token'].isin(train_scenes)].reset_index(drop=True)  # Seleciona os dados de treino com base nas cenas de treino e reseta o índice
test_data = data[data['scene_token'].isin(test_scenes)].reset_index(drop=True)  # Seleciona os dados de teste com base nas cenas de teste e reseta o índice


# Define todos os Hiperparâmetros que serão testados

In [ ]:
# Hiperparâmetros a serem testados
sequence_lengths = [4, 5, 6, 7]  # Define uma lista de comprimentos de sequência a serem testados, representando o número de passos de tempo de entrada
units_list = [256, 32, 64, 128, 512]  # Define uma lista de tamanhos de unidades para as camadas LSTM, representando a quantidade de neurônios por camada
dropout_rates = [0.1, 0.2]  # Define uma lista de taxas de dropout para regularização, ajudando a reduzir o overfitting
epochs_list = [10, 30, 200]  # Define uma lista de valores para o número de épocas de treinamento
activations = ['swish', 'relu', 'linear', 'selu', LeakyReLU(), 'tanh']  # Define uma lista de funções de ativação a serem testadas nas camadas ocultas
learning_rates = [0.001]  # Define uma lista com a taxa de aprendizado do otimizador, controlando a velocidade de ajuste dos pesos

# Definir colunas de features e target
base_feature_cols = ['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']  # Colunas de características de entrada, representando coordenadas de bounding boxes
target_cols = ['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']  # Colunas de alvos que o modelo irá prever


# Aplicação do método RNN - LSTM

In [ ]:
# Função para criar o modelo LSTM Sequencial
def create_lstm_model(sequence_length, num_features, units, dropout_rate, activation, learning_rate):
    model = Sequential()  # Inicializa o modelo sequencial
    model.add(Input(shape=(sequence_length, num_features)))  # Adiciona a camada de entrada com o comprimento da sequência e número de características
    model.add(LSTM(units, activation=activation))  # Adiciona uma camada LSTM com o número de unidades e função de ativação especificada
    model.add(Dropout(dropout_rate))  # Adiciona uma camada de Dropout para regularização
    model.add(Dense(4, activation='linear'))  # Adiciona uma camada densa de saída para prever [min_x, max_x, min_y, max_y]
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])  # Compila o modelo com otimizador Adam, função de perda e métrica MAE
    return model  # Retorna o modelo configurado

# Função para calcular o IoU de forma vetorizada
def calcular_iou_vetorizado(bbox_real, bbox_predito):
    xA = np.maximum(bbox_real[:, 0], bbox_predito[:, 0])  # Calcula a coordenada x do canto superior esquerdo da interseção
    yA = np.maximum(bbox_real[:, 2], bbox_predito[:, 2])  # Calcula a coordenada y do canto superior esquerdo da interseção
    xB = np.minimum(bbox_real[:, 1], bbox_predito[:, 1])  # Calcula a coordenada x do canto inferior direito da interseção
    yB = np.minimum(bbox_real[:, 3], bbox_predito[:, 3])  # Calcula a coordenada y do canto inferior direito da interseção

    interArea = np.maximum(0, xB - xA) * np.maximum(0, yB - yA)  # Calcula a área da interseção garantindo que seja não-negativa
    boxA_area = (bbox_real[:, 1] - bbox_real[:, 0]) * (bbox_real[:, 3] - bbox_real[:, 2])  # Calcula a área da bounding box real
    boxB_area = (bbox_predito[:, 1] - bbox_predito[:, 0]) * (bbox_predito[:, 3] - bbox_predito[:, 2])  # Calcula a área da bounding box predita

    epsilon = 1e-7  # Define uma pequena constante para evitar divisão por zero
    iou = interArea / (boxA_area + boxB_area - interArea + epsilon)  # Calcula o IoU usando a área de interseção e as áreas das caixas
    return iou  # Retorna o valor do IoU

best_models = []  # Inicializa uma lista para armazenar os melhores modelos encontrados


# Itera todos os Hiperparâmetros entre si, NÃO escolhe aleatóriamente, testa todas as combinações.

In [ ]:
# Loop sobre cada combinação de hiperparâmetros
for (sequence_length, units, dropout_rate, learning_rate, activation, epochs) in itertools.product(sequence_lengths, units_list, dropout_rates, learning_rates, activations, epochs_list):
    activation_name = activation if isinstance(activation, str) else activation.__class__.__name__  # Obtém o nome da função de ativação, verificando se é uma string ou uma classe
    print(f'\nTreinando com hiperparâmetros: sequence_length={sequence_length}, units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, activation={activation_name}, epochs={epochs}')

    # Atualiza feature_cols
    feature_cols = base_feature_cols.copy()  # Cria uma cópia de base_feature_cols para evitar modificações na lista original

    # Prepara os dados de treino e teste
    X_train, y_train, _ = prepare_data_for_prediction(train_data, sequence_length, feature_cols, target_cols)  # Gera dados de treino
    X_test, y_test, test_metadata = prepare_data_for_prediction(test_data, sequence_length, feature_cols, target_cols)  # Gera dados de teste

    # Verifica se há dados suficientes
    if X_train.shape[0] == 0 or X_test.shape[0] == 0:
        print("Dados insuficientes para o sequence_length especificado. Pulando esta combinação.")
        continue  # Pula a iteração se não houver dados suficientes

    # Garante que os tipos dos dados sejam float32
    X_train = X_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    y_test = y_test.astype(np.float32)

    # Verifica se há valores NaN
    if np.isnan(X_train).any() or np.isnan(y_train).any() or np.isnan(X_test).any() or np.isnan(y_test).any():
        print("Dados contêm NaNs. Pulando esta combinação.")
        continue  # Pula a iteração se houver valores NaN

    # Normaliza as coordenadas das bounding boxes
    scaler = MinMaxScaler()  # Inicializa o normalizador MinMaxScaler
    num_features = X_train.shape[2]  # Obtém o número de características
    bbox_indices = [feature_cols.index(col) for col in target_cols]  # Determina os índices das colunas das bounding boxes

    # Achata X_train para ajustar o scaler
    X_train_flat = X_train.reshape(-1, num_features)  # Converte X_train para 2D para ajustar o normalizador

    # Combina as coordenadas de X_train e y_train para ajustar o scaler
    all_bbox_train = np.vstack([X_train_flat[:, bbox_indices], y_train])  # Empilha os valores de treino para ajuste do scaler
    scaler.fit(all_bbox_train)  # Ajusta o normalizador com os dados empilhados

    # Transforma X_train
    X_train_flat[:, bbox_indices] = scaler.transform(X_train_flat[:, bbox_indices])  # Normaliza as coordenadas de bounding box em X_train
    X_train = X_train_flat.reshape(-1, sequence_length, num_features)  # Restaura a forma original de X_train

    # Transforma y_train
    y_train = scaler.transform(y_train)  # Normaliza as coordenadas de bounding box em y_train

    # Transforma X_test
    X_test_flat = X_test.reshape(-1, num_features)  # Converte X_test para 2D para normalização
    X_test_flat[:, bbox_indices] = scaler.transform(X_test_flat[:, bbox_indices])  # Normaliza as coordenadas de bounding box em X_test
    X_test = X_test_flat.reshape(-1, sequence_length, num_features)  # Restaura a forma original de X_test

    # Transforma y_test
    y_test = scaler.transform(y_test)  # Normaliza as coordenadas de bounding box em y_test

    # Constrói o modelo
    lstm_model = create_lstm_model(sequence_length, num_features, units, dropout_rate, activation, learning_rate)  # Cria o modelo LSTM com os hiperparâmetros especificados

    # Implementa EarlyStopping
    early_stopping = EarlyStopping(
        monitor='val_loss',  # Monitora a perda de validação para determinar o momento de parada
        patience=10,  # Define a paciência para o número de épocas sem melhora
        restore_best_weights=True  # Restaura os pesos do modelo para o melhor ponto de validação
    )

    # Treina o modelo
    lstm_model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping],
        verbose=0  # Muda para 1 para exibir o progresso do treinamento
    )

    # Faz previsões
    predictions = lstm_model.predict(X_test)  # Gera predições com o modelo treinado

    # Desfaz a normalização para calcular o IoU
    predictions_unscaled = scaler.inverse_transform(predictions)  # Reverte a normalização para as previsões
    y_test_unscaled = scaler.inverse_transform(y_test)  # Reverte a normalização para os alvos de teste

    # Garante que test_metadata tenha o mesmo comprimento que as predições
    correct_length = len(y_test_unscaled)
    if len(test_metadata) > correct_length:
        test_metadata = test_metadata.iloc[:correct_length]
    elif len(test_metadata) < correct_length:
        # Trunca as predições para corresponder ao metadata
        predictions_unscaled = predictions_unscaled[:len(test_metadata)]
        y_test_unscaled = y_test_unscaled[:len(test_metadata)]
        correct_length = len(test_metadata)

    # Calcula o IoU
    bboxes_real = y_test_unscaled  # Define as bounding boxes reais
    bboxes_pred = predictions_unscaled  # Define as bounding boxes preditas
    ious = calcular_iou_vetorizado(bboxes_real, bboxes_pred)  # Calcula o IoU para cada bounding box
    mean_iou = np.mean(ious)  # Calcula o IoU médio
    print(f'IoU Médio do modelo LSTM: {mean_iou:.2f}')  # Exibe o IoU médio

    best_models.append({
        'model': lstm_model,
        'iou': mean_iou,
        'config': {
            'sequence_length': sequence_length,
            'units': units,
            'dropout_rate': dropout_rate,
            'epochs': epochs,
            'activation_name': activation_name,
            'learning_rate': learning_rate
        },
        'scaler': scaler,
        'test_inputs': X_test,
        'test_outputs': y_test,
        'test_metadata': test_metadata
    })  # Armazena o modelo e suas configurações caso seja um dos melhores

# Ordena os modelos pelo maior IoU e mantém os 10 melhores
best_models = sorted(best_models, key=lambda x: x['iou'], reverse=True)[:10]

# Imprime as 10 melhores configurações encontradas
for idx, best_model in enumerate(best_models, start=1):
    print(f'\nModelo {idx}:')
    print(f'Sequence Length: {best_model["config"]["sequence_length"]}')
    print(f'Units: {best_model["config"]["units"]}')
    print(f'Dropout Rate: {best_model["config"]["dropout_rate"]}')
    print(f'Epochs: {best_model["config"]["epochs"]}')
    print(f'Activation Name: {best_model["config"]["activation_name"]}')
    print(f'Learning Rate: {best_model["config"]["learning_rate"]}')
    print(f'IoU: {best_model["iou"]:.2f}')


# Cálculo de movimentos das bounding boxes para análise de deslocamento de objetos

In [ ]:
# Função para analisar movimentos das bounding boxes
def analyze_bbox_movements(df):
    # Ordena o DataFrame por instance_token, camera e timestamp para garantir a ordem correta dos dados
    df = df.sort_values(by=['instance_token', 'camera', 'timestamp']).reset_index(drop=True)
    distances = []  # Inicializa uma lista para armazenar as distâncias calculadas entre bounding boxes consecutivas

    unique_instances = df['instance_token'].unique()  # Obtém uma lista única de instâncias (objetos) no DataFrame

    for instance in unique_instances:
        instance_data = df[df['instance_token'] == instance].reset_index(drop=True)  # Filtra os dados para a instância atual e reseta o índice
        cameras = instance_data['camera'].unique()  # Obtém uma lista única de câmeras para a instância atual

        for cam in cameras:
            cam_data = instance_data[instance_data['camera'] == cam].reset_index(drop=True)  # Filtra os dados para a câmera atual e reseta o índice
            bboxes = cam_data[['bbox_min_x', 'bbox_min_y', 'bbox_max_x', 'bbox_max_y']].values  # Extrai as coordenadas das bounding boxes como array

            for i in range(1, len(bboxes)):
                bbox_prev = bboxes[i - 1]  # Bounding box anterior
                bbox_curr = bboxes[i]  # Bounding box atual

                # Calcula os centros das bounding boxes
                center_prev = [(bbox_prev[0] + bbox_prev[2]) / 2, (bbox_prev[1] + bbox_prev[3]) / 2]  # Centro da bounding box anterior
                center_curr = [(bbox_curr[0] + bbox_curr[2]) / 2, (bbox_curr[1] + bbox_curr[3]) / 2]  # Centro da bounding box atual

                # Calcula a distância euclidiana entre os centros
                dist = np.linalg.norm(np.array(center_curr) - np.array(center_prev))  # Calcula a distância entre os centros das bounding boxes
                distances.append(dist)  # Adiciona a distância à lista de distâncias

    return distances  # Retorna a lista de distâncias calculadas

# Calcula as distâncias no conjunto de treinamento
distances = analyze_bbox_movements(train_data)  # Calcula as distâncias entre movimentos das bounding boxes no conjunto de treino

# Converte as distâncias para um array NumPy
distances = np.array(distances)  # Converte a lista de distâncias para um array NumPy

# Calcula resumos estatísticos
mean_distance = np.mean(distances)  # Calcula a distância média
median_distance = np.median(distances)  # Calcula a mediana das distâncias
std_distance = np.std(distances)  # Calcula o desvio padrão das distâncias
min_distance = np.min(distances)  # Calcula a distância mínima
max_distance = np.max(distances)  # Calcula a distância máxima

# Imprime os resumos estatísticos
print("\nResumos Estatísticos dos Movimentos das Bounding Boxes:")
print(f"Distância Média: {mean_distance:.2f}")
print(f"Mediana da Distância: {median_distance:.2f}")
print(f"Desvio Padrão: {std_distance:.2f}")
print(f"Distância Mínima: {min_distance:.2f}")
print(f"Distância Máxima: {max_distance:.2f}")


# Algoritmo de rastreamento simples para identificação e acompanhamento de objetos por câmera

In [ ]:
# Função de Rastreamento Simples
def simple_tracker(pred_bboxes, timestamps, cameras, max_distance=median_distance):
    pred_ids_array = np.zeros(len(pred_bboxes), dtype=int)  # Inicializa um array para armazenar os IDs preditos, preenchido com zeros
    next_id = 1  # Inicializa o próximo ID a ser atribuído
    active_tracks_per_camera = {}  # Dicionário para armazenar as tracks ativas separadas por câmera

    # Inicializa tracks ativos para cada câmera
    for cam in np.unique(cameras):
        active_tracks_per_camera[cam] = {}  # Cria um dicionário vazio para cada câmera

    # Ordena índices para processar em ordem de timestamp e câmera
    sort_idx = np.lexsort((timestamps, cameras))  # Ordena os índices para garantir a sequência correta de processamento
    pred_bboxes_sorted = pred_bboxes[sort_idx]  # Ordena as bounding boxes preditas com base nos índices ordenados
    timestamps_sorted = timestamps[sort_idx]  # Ordena os timestamps com base nos índices ordenados
    cameras_sorted = cameras[sort_idx]  # Ordena as câmeras com base nos índices ordenados

    for idx in range(len(pred_bboxes_sorted)):
        cam = cameras_sorted[idx]  # Define a câmera atual
        t = timestamps_sorted[idx]  # Define o timestamp atual
        bbox = pred_bboxes_sorted[idx]  # Define a bounding box predita atual
        idx_original = sort_idx[idx]  # Índice original do dado antes da ordenação

        active_tracks = active_tracks_per_camera[cam]  # Recupera as tracks ativas para a câmera atual

        # Se não houver tracks ativas, cria uma nova track
        if not active_tracks:
            pred_ids_array[idx_original] = next_id  # Atribui um novo ID à bounding box atual
            active_tracks[next_id] = {'bbox': bbox, 'timestamp': t}  # Adiciona a nova track ao dicionário
            next_id += 1  # Incrementa o próximo ID disponível
        else:
            # Calcula distâncias entre a bounding box atual e as bounding boxes das tracks ativas
            track_ids = list(active_tracks.keys())  # Obtém os IDs das tracks ativas
            track_bboxes = np.array([active_tracks[tid]['bbox'] for tid in track_ids])  # Extrai as bounding boxes das tracks ativas

            distances = cdist(track_bboxes, [bbox], metric='euclidean').flatten()  # Calcula a distância euclidiana entre a bounding box atual e as tracks ativas
            min_dist_idx = np.argmin(distances)  # Obtém o índice da menor distância
            min_dist = distances[min_dist_idx]  # Define o valor da menor distância

            # Se a menor distância estiver abaixo do limite, associa a bounding box atual à track correspondente
            if min_dist < max_distance:
                tid = track_ids[min_dist_idx]  # Obtém o ID da track com menor distância
                pred_ids_array[idx_original] = tid  # Atribui o ID da track à bounding box atual
                active_tracks[tid] = {'bbox': bbox, 'timestamp': t}  # Atualiza a track ativa com a nova posição da bounding box
            else:
                # Se a distância for maior que o limite, cria uma nova track
                pred_ids_array[idx_original] = next_id  # Atribui um novo ID à bounding box atual
                active_tracks[next_id] = {'bbox': bbox, 'timestamp': t}  # Adiciona a nova track ao dicionário de tracks ativas
                next_id += 1  # Incrementa o próximo ID disponível

    return pred_ids_array  # Retorna o array com os IDs preditos para cada bounding box


# Aplicação das métricas MOTA, IDS, FN, FP, e demais métricas relevantes para análise dos resultados.

In [ ]:
# Função para aplicar métricas
def apply_mot_metrics(df):
    # Cria um mapeamento único para cada combinação de instance_token e camera
    df['gt_id'] = df.apply(lambda row: f"{row['instance_token']}_{row['camera']}", axis=1)  # Cria uma coluna 'gt_id' que combina instance_token e câmera para identificação única

    # Mapeia cada combinação única para um ID numérico único
    unique_gt_ids = df['gt_id'].unique()  # Obtém todos os IDs de ground truth únicos
    id_mapping_gt = {token: idx + 1 for idx, token in enumerate(unique_gt_ids)}  # Associa cada gt_id a um número começando em 1
    df['gt_id_num'] = df['gt_id'].map(id_mapping_gt)  # Adiciona a coluna 'gt_id_num' com IDs numéricos

    # Inicializa o acumulador com auto_id=False
    acc = mm.MOTAccumulator(auto_id=False)  # Inicializa o acumulador para as métricas MOT sem auto_id

    # Atribui um frameid global sequencial para cada timestamp único
    unique_timestamps = sorted(df['timestamp'].unique())  # Ordena os timestamps únicos
    timestamp_to_frameid = {timestamp: idx for idx, timestamp in enumerate(unique_timestamps)}  # Mapeia timestamps para IDs de frames
    df['frameid'] = df['timestamp'].map(timestamp_to_frameid)  # Adiciona a coluna 'frameid' com IDs de frames sequenciais

    # Ordena o DataFrame por frameid
    df = df.sort_values(by=['frameid'])  # Garante que o DataFrame esteja ordenado por frameid

    # Agrupa detecções por frameid
    grouped = df.groupby('frameid')

    for frameid, group in grouped:
        # IDs e bounding boxes de ground truth
        gt_ids = group['gt_id_num'].tolist()  # Lista de IDs ground truth para o frame atual
        gt_bboxes = group[['true_min_x', 'true_min_y', 'true_max_x', 'true_max_y']].values  # Bounding boxes ground truth

        # IDs e bounding boxes preditos
        pred_ids_frame = group['pred_id'].tolist()  # Lista de IDs preditos para o frame atual
        pred_bboxes_frame = group[['pred_min_x', 'pred_min_y', 'pred_max_x', 'pred_max_y']].values  # Bounding boxes preditas

        # Converte IDs para strings
        gt_ids = [str(id) for id in gt_ids]
        pred_ids_frame = [str(id) for id in pred_ids_frame]

        # Verifica se há duplicidades de gt_id no mesmo frameid
        duplicates = group.duplicated(subset=['gt_id_num'])
        if duplicates.any():
            print(f"Removendo {duplicates.sum()} duplicatas de gt_id no frameid {frameid}.")
            group = group.drop_duplicates(subset=['gt_id_num'], keep='first')
            gt_ids = group['gt_id_num'].tolist()
            gt_bboxes = group[['true_min_x', 'true_min_y', 'true_max_x', 'true_max_y']].values
            distances = mm.distances.iou_matrix(gt_bboxes, pred_bboxes_frame, max_iou=0.5)  # Calcula IoU com limite máximo

        # Calcula a matriz de distâncias (usando IoU)
        distances = mm.distances.iou_matrix(gt_bboxes, pred_bboxes_frame, max_iou=0.5)

        # Atualiza o acumulador com as detecções deste frame
        acc.update(
            gt_ids,
            pred_ids_frame,
            distances,
            frameid=frameid
        )

    return acc  # Retorna o acumulador com os dados de métricas


metrics_results = []

for idx, best_model in enumerate(best_models, start=1):
    print(f'\nProcessando Métricas para Modelo {idx}...')

    lstm_model = best_model['model']
    scaler = best_model['scaler']
    X_test = best_model['test_inputs']
    y_test = best_model['test_outputs']
    test_metadata = best_model['test_metadata']

    # Faz previsões com o modelo atual
    predictions = lstm_model.predict(X_test)

    # Desfaz a normalização para calcular métricas
    predictions_unscaled = scaler.inverse_transform(predictions)  # Reverte a normalização das previsões
    y_test_unscaled = scaler.inverse_transform(y_test)  # Reverte a normalização dos valores de teste

    # Garante que test_metadata tenha o mesmo comprimento que as predições
    correct_length = len(y_test_unscaled)
    if len(test_metadata) > correct_length:
        test_metadata = test_metadata.iloc[:correct_length]
    elif len(test_metadata) < correct_length:
        predictions_unscaled = predictions_unscaled[:len(test_metadata)]
        y_test_unscaled = y_test_unscaled[:len(test_metadata)]
        correct_length = len(test_metadata)

    # Cria DataFrame para métricas MOT
    df_results = pd.DataFrame({
        'instance_token': test_metadata['instance_token'],
        'timestamp': test_metadata['timestamp'],
        'camera': test_metadata['camera'],
        'scene_token': test_metadata['scene_token'],
        'true_min_x': y_test_unscaled[:, 0],
        'true_max_x': y_test_unscaled[:, 1],
        'true_min_y': y_test_unscaled[:, 2],
        'true_max_y': y_test_unscaled[:, 3],
        'pred_min_x': predictions_unscaled[:, 0],
        'pred_max_x': predictions_unscaled[:, 1],
        'pred_min_y': predictions_unscaled[:, 2],
        'pred_max_y': predictions_unscaled[:, 3]
    })

    # Adiciona colunas necessárias para o tracker
    df_results['timestamp'] = df_results['timestamp'].astype(int)
    pred_bboxes = df_results[['pred_min_x', 'pred_min_y', 'pred_max_x', 'pred_max_y']].values
    timestamps = df_results['timestamp'].values
    cameras = df_results['camera'].values

    # Aplica o tracker para atribuir IDs preditos
    pred_ids = simple_tracker(pred_bboxes, timestamps, cameras, max_distance=median_distance)
    df_results['pred_id'] = pred_ids

    # Aplica métricas MOT
    accumulator = apply_mot_metrics(df_results)

    # Cria um manipulador de métricas para gerar métricas dos dados acumulados
    mh = mm.metrics.create()

    # Calcula várias métricas
    summary = mh.compute(
        accumulator,
        metrics=['num_frames', 'num_switches', 'mota',  'idfp', 'idfn', 'motp', 'idf1', 'idp', 'idr', 'recall', 'precision'],
        name=f'Model_{idx}'
    )

    # Adiciona resultado ao conjunto de resultados
    metrics_results.append(summary)

    print(f'Métricas calculadas para Modelo {idx}.')

# Verifica se há resultados para serem concatenados e salva em um arquivo CSV
if metrics_results:
    metrics_summary_df = pd.concat(metrics_results, axis=0).reset_index(drop=True)
    metrics_summary_df.to_csv('mot_metrics_summary.csv', index=False)
    print('\nMétricas MOT salvas em mot_metrics_summary.csv')
else:
    print('Nenhuma métrica calculada para os modelos selecionados.')

# Imprime o resumo das métricas
print(metrics_summary_df)



Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=swish, epochs=10
IoU Médio do modelo LSTM: 0.05

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=swish, epochs=30
IoU Médio do modelo LSTM: 0.26

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=swish, epochs=200
IoU Médio do modelo LSTM: 0.23

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=relu, epochs=10
IoU Médio do modelo LSTM: 0.14

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=relu, epochs=30
IoU Médio do modelo LSTM: 0.19

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=relu, epochs=200
IoU Médio do modelo LSTM: 0.19

Treinando com hiperparâmetros: sequence_